In [ ]:
import os

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
!pip install python-docx
import docx
!pip install PyPDF2
import PyPDF2

!pip install transformers datasets

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)
# # note: the path may be different for others.
# os.chdir('/content/drive/MyDrive/DAP Group 6: Fin Scanners/statements')

In [ ]:
!ls

Applications                    monetary20220615a1_summary.docx
Desktop                         monetary20220727a1.pdf
Developer                       monetary20220727a1_summary.docx
Documents                       monetary20220921a1.pdf
Downloads                       monetary20220921a1_summary.docx
Library                         monetary20221102a1.pdf
Movies                          monetary20221102a1_summary.docx
Music                           monetary20221214a1.pdf
Pictures                        monetary20221214a1_summary.docx
Postman                         news_summary.csv
Public                          nltk_data
Sites                           open
Summary 20220126a1.docx         opt
Summary 20220504a1.docx         pandas
Summary 20220615a1.docx         refsum20220126.docx
Summary 20220727a1.docx         refsum20220504.docx
Summary 20220921a1.docx         refsum20220615.docx
Summary 20221102a1.docx         refsum20220727.docx
Summary 20221214a1.docx         refsum20220921.do

# create a function to preprocess the statement to be fed into the model

In [ ]:
def clean_monetary_pdf(monetary_pdf, by="para"):
  # takes in 2 parameters: first is the original statement;
  # second parameter is "para" (for split by paragraphs) or  "sen" (for split by first sentences)

  # opening the pdf
  monetary_pdf = open(monetary_pdf, "rb")
  monetary_pdf = PyPDF2.PdfReader(monetary_pdf)

  # chunking all the text in the pdf together
  text = ""
  for page in monetary_pdf.pages:
    text+=page.extract_text()
    if "-0-" in page:
      break

  # create a header variable which we will need to take out if present
  header = ""
  start = text.find("For release")
  end = text.find("22")
  header = text[start:end+2]

  text = text.split("\n")

  # split by paragraph
  para_summary_input = []
  current_para = ""

  for i,sentence in enumerate(text):
    if "Voting" in sentence:
      break
    elif "-2-" in sentence or "(more)" in sentence:
      continue
    # make sure there's no headers in the sentence
    sentence = sentence.replace(header, "")
    # if first word is a " ", that means it's a start of a new paragraph! (not the same for every text!)
    # if the sentence starts with a capital letter and the prevoious letter is a " " then it's also a new para! (idk how that works)
    # push the current para into the summary and reset current_para
    # print(sentence)
    if sentence[0] == " " or (sentence[0].isupper() and text[i-1][-1] == " "):
      if current_para.strip():
        para_summary_input.append(current_para.strip())
      current_para = sentence
    else:
        current_para += sentence
  # add in the leftovers
  if current_para:
    para_summary_input.append(current_para)

  if by == "para":
    return para_summary_input

  # split and get the first sentences
  if by == "sen":
    first_n_summary = ""
    for para in para_summary_input:      
      first_n_summary += para.split(". ")[0] + ". "
    
    return first_n_summary

(sort of\) works for everything except for the statements in Jun, Jul and Sep

In [ ]:
by_para_mon202212 = ['Recent indicators point to modest growth in spending and production.  Job gains have been robust in recent months, and the unemployment rate has remained low.  Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic, higher food and energy prices, and broader price pressures.', 'Russia’s war against Ukraine is causing tremendous human and economic hardship.  The war and related e vents are contributing to upward pressure on inflation and are weighing on global economic activity.  The Committee is highly attentive to inflation risks.', 'The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run.  In support of these goals, the Committee decided to raise the target range for the federal funds rate to 4 -1/4 to 4- 1/2 percent.  The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary policy that is sufficiently restrictive to return inflation to 2 percent over time.  In determining the pace of future increases in the target range, the Committee will take into account the cumulative tightening of monetary policy, the lags  with which monetary policy affects economic activity and inflation, and economic and financial developments.  In addition, the Committee will continue reducing its holdings of Treasury securities and agency debt and agency mortgage -backed securities, as d escribed in the Plans for Reducing the Size of the Federal Reserve’s Balance Sheet that were issued in May.  The Committee is strongly committed to returning inflation to its 2 percent objective.', 'In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook.  The Committee would be prepared to adjust the stance of monetary policy as appropriate if ris ks emerge that could impede the attainment of the Committee’s goals.  The Committee’s assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation pressures and inflation expectations, and financial and international developments.   ']
by_sen_mon202212 = "Recent indicators point to modest growth in spending and production. Russia’s war against Ukraine is causing tremendous human and economic hardship. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. "

In [ ]:
by_para_mon202211 = ['Recent indicators point to modest growth in spending and production.  Job gains have been robust in recent months, and the unemployment rate has remained low.  Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic, higher food and energy prices, and broader price pressures .', 'Russia’s war against Ukraine is causing tremendous human and economic hardship.  The war and related  events are creating additional upward pressure on inflation and are weighing on global economic activity.  The Committee is highly attentive to inflation risks.', 'The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent ov er the longer run.  In support of these goals, the Committee decided to raise the target range for the federal funds rate to 3 -3/4 to 4 percent.  The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stance of monetary policy that is sufficiently restrictive to return inflation to 2 percent over time .  In determining the pace of future increases in the target range, the Committee will take into account the cumulative tightening of monetary policy, the lags with which monetary policy affects economic activity and inflation, and economic and financial developments.  In addition, the Committee will continue reducing its holdings of Treasury securities and agency debt and agency mortgage -backed securities, as described in the Plans for Reducing the Size of the Federal Reserve’s Balance Sheet that were issued in May.  The Committee is strongly committed to returning inflation to its 2 percent objective.', ' In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook.  The Committee would be prepared to adjust the stance of monetary policy as appropriate if ris ks emerge that could impede the attainment of the Committee’s goals.  The Committee’s assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation pressures and inflation expectations, and financial and international developments.   ']
by_sen_mon202211 = "Recent indicators point to modest growth in spending and production. Russia’s war against Ukraine is causing tremendous human and economic hardship. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent ov er the longer run.  In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. "

In [ ]:
by_para_mon202205 = ['Although overall economic activity edged down in the first quarter, household spending and business fixed investment remained strong.  Job gains have been robust  in recent months, and the unemployment rate has declined substantially.  Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic, higher energy prices, and broader price pressures.', 'The invasion of Ukraine by Russia is c ausing tremendous human and economic hardship.  The implications for the U.S. economy are highly uncertain.  The invasion and related events are creating  additional upward pressure on inflation and are likely to weigh on economic activity.  In addition, CO VID- related lockdowns in China are likely to exacerbate supply chain disruptions.  The Committee is highly attentive to inflation risks.', 'The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the longer run.  With appropriate firming in the stance of monetary policy, the Committee expects inflation to return to its 2  percent objective and the labor market to remain strong.  In support of these goals, the Committee decided to raise the target range for the federal funds rate to 3/4 to 1 percent and anticipates that ongoing increases in the target range will be appropriate.  In addition, the Committee decided  to begin reducing its holdings of Treasury securities and agency debt and agency mortgage -backed securities on June 1, as described in the Plans for Reducing the Size of the Federal Reserve’s Balance Sheet that were issued in conjunction with this statement.', 'In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic', '  outlook.  The Committee would be prepared to adjust the stance of monetary policy as appropriate if risks emerge that could impede the attainment of the Committee’s goals.  The Committee’s assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation pressures and inflation expectations, and financial and international developments.  ']
by_sen_mon202205 = "Although overall economic activity edged down in the first quarter, household spending and business fixed investment remained strong. The invasion of Ukraine by Russia is c ausing tremendous human and economic hardship. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the longer run. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic.   outlook. "

In [ ]:
by_para_mon202201 = ['Indicators of economic activity and empl oyment have continued to strengthen.', 'The sectors most adversely affected by th e pandemic have improved in recent months but are being affected by the recent sharp rise in COVID-19 cases.  Job gains have been solid in recent months, and the unemployment rate has declined substantially.  Supply and demand imbalances related to the pandemic and the reopening of the economy have continued to contribute to el evated levels of inflation.  Overall financial conditions remain accommodative, in part reflecting policy measures to support the economy and the flow of cred it to U.S. households and businesses.', 'The path of the economy continues to depe nd on the course of the virus.  Progress on vaccinations and an easing of supply cons traints are expected to support continued gains in economic activity and employment as well as a reduction in inflation.  Risks to the economic outlook re main, including from new va riants of the virus.', 'The Committee seeks to achieve maximum em ployment and inflation at the rate of 2 percent over the longer run.  In s upport of these goals, the Committee decided to keep the target range for the federal funds rate  at 0 to 1/4 percent.  With inflation well above 2 percent and a strong labor market , the Committee exp ects it will soon be appropriate to raise the target range for the federal funds rate.   The Committee decided to continue to redu ce the monthly pace of its net asset purchases, bringing them to an end in early March.  Beginni ng in February, the Committee will increase its holdings of Treasury securities by at least $20 billion per month and of agency mortgage ‑backed securities by at l east $10 billion per month.  The Federal Reserve’s ongoing purchases and  holdings of securities will con tinue to foster smooth market functioning and accommodative financial condi tions, thereby supporting the flow of credit to households and businesses.', 'In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of  incoming information for the economic outlook.  The Committee would be prepared to  adjust the stance of monetary policy as appropriate if risks emer ge that could impede the at tainment of the Committee’s goals.  The Committee’s assessments will take into account a wide range of information, including readi ngs on public health, labor market conditions, inflation pressures and inflation exp ectations, and financial and international developments. ']
by_sen_mon202201 = "Indicators of economic activity and empl oyment have continued to strengthen.. The sectors most adversely affected by th e pandemic have improved in recent months but are being affected by the recent sharp rise in COVID-19 cases. The path of the economy continues to depe nd on the course of the virus. The Committee seeks to achieve maximum em ployment and inflation at the rate of 2 percent over the longer run. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of  incoming information for the economic outlook. "

In [ ]:
# print(by_para_mon202212)
# print(by_sen_mon202212)

In [ ]:
# print(by_para_mon202211)
# print(by_sen_mon202211)

In [ ]:
# print(by_para_mon202205)
# print(by_sen_mon202205)

In [ ]:
# print(by_para_mon202201)
# print(by_sen_mon202201)

In [ ]:
by_para_mon = [by_para_mon202212, by_para_mon202211, by_para_mon202205, by_para_mon202201]
by_sen_mon = [by_sen_mon202212, by_sen_mon202211, by_sen_mon202205, by_sen_mon202201]

# code to extract our human-summarisation from the docx into the model

In [ ]:
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return ' '.join(fullText)

In [ ]:
# given the way i write the for-loop later on, the order should stay like that

human202212 = getText("refsum20221214.docx")
human202211 = getText("refsum20221102.docx")
human202205 = getText("refsum20220504.docx")
human202201 = getText("refsum20220126.docx")
tarasha202212 = getText("Summary 20221214a1.docx")
tarasha202211 = getText("Summary 20221102a1.docx")
tarasha202205 = getText("Summary 20220504a1.docx")
tarasha202201 = getText("Summary 20220126a1.docx")
arjun202212 = getText("monetary20220126a1_summary.docx")
arjun202211 = getText("monetary20221102a1_summary.docx")
arjun202205 = getText("monetary20220504a1_summary.docx")
arjun202201 = getText("monetary20220126a1_summary.docx")

by_human = [human202212, human202211, human202205, human202201]
by_tarasha = [tarasha202212, tarasha202211, tarasha202205, tarasha202201]
by_arjun = [arjun202212, arjun202211, arjun202205, arjun202201]

# Evaluating baseline model

In [ ]:
!pip install transformers
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
!pip install sentencepiece

## Based on ROUGE, BLEU, BLANC

In [ ]:
!pip install rouge
from rouge import Rouge
rouge = Rouge()

def calc_rouge_scores(model_summary, reference_summary):
    return rouge.get_scores(model_summary, reference_summary)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
from nltk.corpus import stopwords
from nltk import download

download('stopwords')
stop_words = stopwords.words('english')

def preprocess(sentence):
  return [w for w in sentence.lower().split() if w not in stop_words]

def preprocess_to_str(sentence):
  sentence = sentence.lower().split()
  result = ""
  for i in sentence:
    if i not in stop_words:
      result += (i + " ")
  return result

[nltk_data] Downloading package stopwords to /Users/yj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_baseline_rouge = {}
baseline_bleu = []

N = len(by_human)
for i in range(N):
  result = calc_rouge_scores(preprocess_to_str(by_sen_mon[i]), preprocess_to_str(by_human[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

  result_bleu = sentence_bleu([preprocess(by_sen_mon[i])], preprocess(by_human[i]))
  baseline_bleu.append(result_bleu)

avg_baseline_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_baseline_rouge)
print(baseline_bleu)

{'rouge-1': {'r': 0.34912712637074417, 'p': 0.35020739064856715, 'f': 0.3478143997283309}, 'rouge-2': {'r': 0.16606624325389321, 'p': 0.1857492936631494, 'f': 0.17428230866688513}, 'rouge-l': {'r': 0.34912712637074417, 'p': 0.35020739064856715, 'f': 0.3478143997283309}}
[5.359624986283916e-155, 0.11169314207209548, 0.22087675159484207, 0.05068034052957636]


/Users/yj/opt/anaconda3/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/yj/opt/anaconda3/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [ ]:
import gensim.downloader as api
wmdmodel = api.load('word2vec-google-news-300')
!pip install pyemd

In [ ]:
from nltk.corpus import stopwords
from nltk import download

download('stopwords')
stop_words = stopwords.words('english')

def preprocess(sentence):
  a = []
  for sen in sentence:
    for word in sen.split(" "):
      if word.lower() not in stop_words:
        a.append(word.lower())
  return a

def preprocess2(sentence):
  return [w for w in sentence.lower().split() if w not in stop_words]

def preprocess_to_str(sentence):
  sentence = sentence.lower().split()
  result = ""
  for i in sentence:
    if i not in stop_words:
      result += (i + " ")
  return result

[nltk_data] Downloading package stopwords to /Users/yj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
avg_dist = 0

for i in range(N):
  reference_summary1 = preprocess2(by_human[i])
  reference_summary2 = preprocess2(by_tarasha[i])
  reference_summary3 = preprocess2(by_arjun[i])
  model_summary = preprocess2(by_sen_mon[i])
  print(f"*** WMD for the {i+1}th summary ***")
  distance1 = wmdmodel.wmdistance(model_summary, reference_summary1)
  print('distance = %.4f' % distance1)
  distance2 = wmdmodel.wmdistance(model_summary, reference_summary2)
  print('distance = %.4f' % distance2)
  distance3 = wmdmodel.wmdistance(model_summary, reference_summary3)
  print('distance = %.4f' % distance3)
  dist = (distance1 + distance2 + distance3) / 3
  avg_dist += dist
  print('average distance = %.4f' % dist + "\n")
print("*** Average WMD from all 4 summaries *** ")
print(avg_dist/4)

*** WMD for the 1th summary ***
distance = 0.8513
distance = 0.6852
distance = 1.0677
average distance = 0.8681

*** WMD for the 2th summary ***
distance = 0.7395
distance = 0.6466
distance = 0.8660
average distance = 0.7507

*** WMD for the 3th summary ***
distance = 0.6600
distance = 0.7038
distance = 0.6973
average distance = 0.6870

*** WMD for the 4th summary ***
distance = 0.9563
distance = 0.8272
distance = 1.0176
average distance = 0.9337

*** Average WMD from all 4 summaries *** 
0.8098764879981596


In [ ]:
# print(preprocess(by_sen_mon[i]))
# print(preprocess(by_human[i]))

In [ ]:
# function to get the combined avg of all 3 results

def avg_rouge_results(all_results):
  N = len(all_results)
  ans = {'rouge-1':{'r': 0, 'p': 0, 'f': 0}, 'rouge-2':{'r': 0, 'p': 0, 'f': 0}, 'rouge-l':{'r': 0, 'p': 0, 'f': 0}}

  # to get the sum of the score
  for i in range(len(all_results)):
    for k, (r, p, f) in all_results[i].items():
      ans[k][r] += all_results[i][k][r]
      ans[k][p] += all_results[i][k][p]
      ans[k][f] += all_results[i][k][f]
  
  # to get the avg score
  for k, (r, p, f) in ans.items():
    ans[k][r] /= N
    ans[k][p] /= N
    ans[k][f] /= N
  return ans

# Getting pegasus model to evaluate the speech that is split by paragraph

In [ ]:
# pegasus for financial data model summary
summarizer1 = pipeline(model = "human-centered-summarization/financial-summarization-pegasus")

tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
model = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")

## Based on rouge score

In [ ]:
# avg score from 1st summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_pegasus_by_para_rouge = {}
pegasus_bleu = []

N = len(by_human)

for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  peg_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    input_ids = tokenizer(para, return_tensors="pt").input_ids
    output = model.generate(
        input_ids, 
        min_length=10,
        max_length=3000,
        num_beams=4, 
        early_stopping=True
    )
    peg_candidate.append(tokenizer.decode(output[0], skip_special_tokens=True))
  peg_cand = " ".join(peg_candidate) # here in peg_cand we have the pegasus generated model
  print(peg_cand)
  
  result = calc_rouge_scores(preprocess_to_str(peg_cand), preprocess_to_str(by_human[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

  result_bleu = sentence_bleu([preprocess(peg_cand)], preprocess(by_human[i]))
  pegasus_bleu.append(result_bleu)

avg_pegasus_by_human_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_pegasus_by_human_rouge)
print(pegasus_bleu)

Economic recovery is gathering pace. Inflation remains elevated, reflecting supply and demand imbalances War against Ukraine is causing tremendous human and economic hardship. The Committee decided to raise the target range for the federal funds rate to 4 -1/4 to 4- 1/2 percent. The Committee will continue to assess the effects of monetary policy on the U.S. economy.
Economic recovery is gathering pace. Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic War, related events are creating additional upward pressure on inflation. The Committee decided to raise the target range for the federal funds rate to 3  to 4 percent. The Committee will continue to assess the effects of monetary policy on the U.S. economy.
Household spending, business fixed investment remain strong. Inflation remains elevated, reflecting supply and demand imbalances The Committee will continue to assess the effects of the invasion on the U.S. economy. The Committee decided to r

In [ ]:
# avg score from 2nd summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_pegasus_by_tarasha_rouge = {}
pegasus_bleu_tarasha = []

N = len(by_tarasha)

for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  peg_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    input_ids = tokenizer(para, return_tensors="pt").input_ids
    output = model.generate(
        input_ids, 
        min_length=10,
        max_length=3000,
        num_beams=4, 
        early_stopping=True
    ) 
    peg_candidate.append(tokenizer.decode(output[0], skip_special_tokens=True))
  peg_cand = " ".join(peg_candidate) # here in peg_cand we have the pegasus generated model
  print(peg_cand)
  
  result = calc_rouge_scores(preprocess_to_str(peg_cand), preprocess_to_str(by_tarasha[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

  result_bleu = sentence_bleu([preprocess(peg_cand)], preprocess(by_tarasha[i]))
  pegasus_bleu_tarasha.append(result_bleu)

avg_pegasus_by_tarasha_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_pegasus_by_tarasha_rouge)
print(pegasus_bleu_tarasha)

Economic recovery is gathering pace. Inflation remains elevated, reflecting supply and demand imbalances War against Ukraine is causing tremendous human and economic hardship. The Committee decided to raise the target range for the federal funds rate to 4 -1/4 to 4- 1/2 percent. The Committee will continue to assess the effects of monetary policy on the U.S. economy.
Economic recovery is gathering pace. Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic War, related events are creating additional upward pressure on inflation. The Committee decided to raise the target range for the federal funds rate to 3  to 4 percent. The Committee will continue to assess the effects of monetary policy on the U.S. economy.
Household spending, business fixed investment remain strong. Inflation remains elevated, reflecting supply and demand imbalances The Committee will continue to assess the effects of the invasion on the U.S. economy. The Committee decided to r

In [ ]:
# avg score from 3rd summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_pegasus_by_arjun_rouge = {}
pegasus_bleu_arjun = []

N = len(by_arjun)

for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  peg_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    input_ids = tokenizer(para, return_tensors="pt").input_ids
    output = model.generate(
        input_ids, 
        min_length=10,
        max_length=3000,
        num_beams=4, 
        early_stopping=True
    )
    peg_candidate.append(tokenizer.decode(output[0], skip_special_tokens=True))
  peg_cand = " ".join(peg_candidate) # here in peg_cand we have the pegasus generated model
  print(peg_cand)
  
  result = calc_rouge_scores(preprocess_to_str(peg_cand), preprocess_to_str(by_arjun[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

  result_bleu = sentence_bleu([preprocess(peg_cand)], preprocess(by_arjun[i]))
  pegasus_bleu_arjun.append(result_bleu)

avg_pegasus_by_ajun_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_pegasus_by_arjun_rouge)
print(pegasus_bleu_arjun)
print(preprocess_to_str(peg_cand), preprocess_to_str(by_arjun[i]))

Economic recovery is gathering pace. Inflation remains elevated, reflecting supply and demand imbalances War against Ukraine is causing tremendous human and economic hardship. The Committee decided to raise the target range for the federal funds rate to 4 -1/4 to 4- 1/2 percent. The Committee will continue to assess the effects of monetary policy on the U.S. economy.
Economic recovery is gathering pace. Inflation remains elevated, reflecting supply and demand imbalances related to the pandemic War, related events are creating additional upward pressure on inflation. The Committee decided to raise the target range for the federal funds rate to 3  to 4 percent. The Committee will continue to assess the effects of monetary policy on the U.S. economy.
Household spending, business fixed investment remain strong. Inflation remains elevated, reflecting supply and demand imbalances The Committee will continue to assess the effects of the invasion on the U.S. economy. The Committee decided to r

In [ ]:
# rouge1r, rouge1p, rouge1f = 0, 0, 0
# rouge2r, rouge2p, rouge2f = 0, 0, 0
# rougelr, rougelp, rougelf = 0, 0, 0
# # avg score for 3rd summaries
# for i in range(N):  # looping through every text(splitted by paragraph) in the list 
#   sshleifer_candidate = []
#   for para in by_para_mon[i]: # looping through every paragraph within that text
#     sshleifer_candidate.append(summarizer2(para, min_length=15,max_length=20)[0]['summary_text'])
  
#   sshleifer_cand = " ".join(sshleifer_candidate)
#   print(sshleifer_cand) 
#   result = calc_rouge_scores(preprocess_to_str(sshleifer_cand), preprocess_to_str(by_arjun[i]))
#   rouge1r += result[0]['rouge-1']['r']
#   rouge1p += result[0]['rouge-1']['p']
#   rouge1f += result[0]['rouge-1']['f']

#   rouge2r += result[0]['rouge-2']['r']
#   rouge2p += result[0]['rouge-2']['p']
#   rouge2f += result[0]['rouge-2']['f']

#   rougelr += result[0]['rouge-l']['r']
#   rougelp += result[0]['rouge-l']['p']
#   rougelf += result[0]['rouge-l']['f']

# avg_bart_by_arjun_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
# print(avg_bart_by_arjun_rouge)

In [ ]:
# # function to average bleu

# def avg_bleu_results(input):
#   result = []
#   result_index = 0
#   for i in range(0, len(input)):
#     for j in range(0, len(input[i])):
#       result.append(input[i][j])
    
#     result_index = 0

#   for i in result:
#     i = i/4

#   return result

In [ ]:
## Combined avg of all 3 summaries
all_peg_rouge = [avg_pegasus_by_human_rouge, avg_pegasus_by_tarasha_rouge, avg_pegasus_by_arjun_rouge]
print(avg_rouge_results(all_peg_rouge))

{'rouge-1': {'r': 0.19321572221441122, 'p': 0.22972972972972971, 'f': 0.20520982774756794}, 'rouge-2': {'r': 0.07768835345670684, 'p': 0.10198170731707316, 'f': 0.08602333800224658}, 'rouge-l': {'r': 0.1874345902456452, 'p': 0.22118272118272117, 'f': 0.19863185287691013}}


## Based on WMD

In [ ]:
# model_summary = preprocess(peg_cand)
# reference_summary1 = preprocess(by_human)
# reference_summary2 = preprocess(by_tarasha)
# reference_summary3 = preprocess(by_arjun)

# distance1 = model.wmdistance(model_summary, reference_summary1)
# print('distance = %.4f' % distance1)
# distance2 = model.wmdistance(model_summary, reference_summary2)
# print('distance = %.4f' % distance2)
# distance3 = model.wmdistance(model_summary, reference_summary3)
# print('distance = %.4f' % distance3)
# avg_dist = (distance1 + distance2 + distance3) / 3
# print('distance = %.4f' % avg_dist)

# Getting sshleifer-Bart model to evaluate the speech that is split by paragraph 

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, AutoModel

In [ ]:
summarizer2 = BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-cnn-12-6")

In [ ]:
summarizer2 = pipeline(model = "sshleifer/distilbart-cnn-12-6")

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

## Based on rouge score

In [ ]:
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
# avg score for 1st summaries
for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  sshleifer_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    sshleifer_candidate.append(summarizer2(para, min_length=15,max_length=20)[0]['summary_text'])
  
  sshleifer_cand = " ".join(sshleifer_candidate)
  print(sshleifer_cand) 
  result = calc_rouge_scores(preprocess_to_str(sshleifer_cand), preprocess_to_str(by_human[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

avg_bart_by_human_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_bart_by_human_rouge)

TypeError: ignored

In [ ]:
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
# avg score for 2nd summaries
for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  sshleifer_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    sshleifer_candidate.append(summarizer2(para, min_length=15,max_length=20)[0]['summary_text'])
  
  sshleifer_cand = " ".join(sshleifer_candidate)
  print(sshleifer_cand) 
  result = calc_rouge_scores(preprocess_to_str(sshleifer_cand), preprocess_to_str(by_tarasha[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

avg_bart_by_tarasha_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_bart_by_tarasha_rouge)

TypeError: ignored

In [ ]:
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
# avg score for 3rd summaries
for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  sshleifer_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    sshleifer_candidate.append(summarizer2(para, min_length=15,max_length=20)[0]['summary_text'])
  
  sshleifer_cand = " ".join(sshleifer_candidate)
  print(sshleifer_cand) 
  result = calc_rouge_scores(preprocess_to_str(sshleifer_cand), preprocess_to_str(by_arjun[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

avg_bart_by_arjun_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_bart_by_arjun_rouge)

In [ ]:
# ## Combined avg of all 3 summaries
all_bart_rouge = [avg_bart_by_human_rouge, avg_bart_by_tarasha_rouge, avg_bart_by_arjun_rouge]

print(avg_rouge_results(all_bart_rouge))

NameError: ignored

## Based on WMD

In [ ]:
# model_summary = preprocess(sshleifer_cand)
# reference_summary1 = preprocess(by_human)
# reference_summary2 = preprocess(by_tarasha)
# reference_summary3 = preprocess(by_arjun)

# distance1 = model.wmdistance(model_summary, reference_summary1)
# print('distance = %.4f' % distance1)
# distance2 = model.wmdistance(model_summary, reference_summary2)
# print('distance = %.4f' % distance2)
# distance3 = model.wmdistance(model_summary, reference_summary3)
# print('distance = %.4f' % distance3)
# avg_dist = (distance1 + distance2 + distance3) / 3
# print('distance = %.4f' % avg_dist)

# Getting T5 model to evaluate the speech that is split by paragraph

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

/Users/yj/opt/anaconda3/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


## Based on rouge score

In [ ]:
# avg score from 1st summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_t5_by_human_rouge = {}

N = len(by_human)

for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  t5_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    text = "summarize: " + para # need to add a "summarize" to tell the T5 what to do
    input_ids = tokenizer.encode(text,return_tensors="pt")
    output_ids = model.generate(
        input_ids, 
        min_length=10,
        max_length=1024
    )
    t5_candidate.append(tokenizer.decode(output_ids[0]))
  t5_cand = " ".join(t5_candidate) # here in t5_cand we have the t5 generated model
  t5_cand = t5_cand.replace("<pad>", "")
  t5_cand = t5_cand.replace("</s>", "")
  print(t5_cand)
  
  result = calc_rouge_scores(preprocess_to_str(t5_cand), preprocess_to_str(by_human[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

avg_t5_by_human_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_t5_by_human_rouge)

 recent indicators point to modest growth in spending and production. inflation remains elevated, reflecting supply and demand imbalances.  the war and related e vents are contributing to upward pressure on inflation. the Committee is highly attentive to inflation risks.  the Committee is strongly committed to returning inflation to its 2 percent objective. the Committee will continue reducing its holdings of Treasury securities and agency debt and agency mortgage -backed securities.  the Committee will monitor the implications of incoming information for the economic outlook. the Committee would be prepared to adjust the stance of monetary policy as appropriate if ris ks emerge that could impede the attainment of the Committee’s goals.
 recent indicators point to modest growth in spending and production. job gains have been robust in recent months, and the unemployment rate has remained low.  the war and related events are creating additional upward pressure on inflation. the Committe

In [ ]:
# avg score from 2nd summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_t5_by_tarasha_rouge = {}

N = len(by_tarasha)

for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  t5_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    text = "summarize: " + para # need to add a "summarize" to tell the T5 what to do
    input_ids = tokenizer.encode(text,return_tensors="pt")
    output_ids = model.generate(
        input_ids, 
        min_length=10,
        max_length=1024
    )
    t5_candidate.append(tokenizer.decode(output_ids[0]))
  t5_cand = " ".join(t5_candidate) # here in t5_cand we have the t5 generated model
  t5_cand = t5_cand.replace("<pad>", "")
  t5_cand = t5_cand.replace("</s>", "")
  print(t5_cand)
  
  result = calc_rouge_scores(preprocess_to_str(t5_cand), preprocess_to_str(by_tarasha[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

avg_t5_by_tarasha_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_t5_by_tarasha_rouge)

 recent indicators point to modest growth in spending and production. inflation remains elevated, reflecting supply and demand imbalances.  the war and related e vents are contributing to upward pressure on inflation. the Committee is highly attentive to inflation risks.  the Committee is strongly committed to returning inflation to its 2 percent objective. the Committee will continue reducing its holdings of Treasury securities and agency debt and agency mortgage -backed securities.  the Committee will monitor the implications of incoming information for the economic outlook. the Committee would be prepared to adjust the stance of monetary policy as appropriate if ris ks emerge that could impede the attainment of the Committee’s goals.
 recent indicators point to modest growth in spending and production. job gains have been robust in recent months, and the unemployment rate has remained low.  the war and related events are creating additional upward pressure on inflation. the Committe

In [ ]:
# avg score from 3rd summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_t5_by_arjun_rouge = {}

N = len(by_arjun)

for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  t5_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    text = "summarize: " + para # need to add a "summarize" to tell the T5 what to do
    input_ids = tokenizer.encode(text,return_tensors="pt")
    output_ids = model.generate(
        input_ids, 
        min_length=10,
        max_length=1024
    )
    t5_candidate.append(tokenizer.decode(output_ids[0]))
  t5_cand = " ".join(t5_candidate) # here in t5_cand we have the t5 generated model
  t5_cand = t5_cand.replace("<pad>", "")
  t5_cand = t5_cand.replace("</s>", "")
  print(t5_cand)
  
  result = calc_rouge_scores(preprocess_to_str(t5_cand), preprocess_to_str(by_arjun[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

avg_t5_by_arjun_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_t5_by_arjun_rouge)

 recent indicators point to modest growth in spending and production. inflation remains elevated, reflecting supply and demand imbalances.  the war and related e vents are contributing to upward pressure on inflation. the Committee is highly attentive to inflation risks.  the Committee is strongly committed to returning inflation to its 2 percent objective. the Committee will continue reducing its holdings of Treasury securities and agency debt and agency mortgage -backed securities.  the Committee will monitor the implications of incoming information for the economic outlook. the Committee would be prepared to adjust the stance of monetary policy as appropriate if ris ks emerge that could impede the attainment of the Committee’s goals.
 recent indicators point to modest growth in spending and production. job gains have been robust in recent months, and the unemployment rate has remained low.  the war and related events are creating additional upward pressure on inflation. the Committe

In [ ]:
# ## Combined avg of all 3 summaries
all_t5_rouge = [avg_t5_by_human_rouge, avg_t5_by_tarasha_rouge, avg_t5_by_arjun_rouge]

print(avg_rouge_results(all_t5_rouge))

{'rouge-1': {'r': 0.37442817453776, 'p': 0.2863023233486943, 'f': 0.3143529495806659}, 'rouge-2': {'r': 0.17612832334760167, 'p': 0.1253047782686958, 'f': 0.1417487529242419}, 'rouge-l': {'r': 0.36308586966707285, 'p': 0.2752064132104455, 'f': 0.30332108974761773}}


## Based on WMD

In [ ]:
# model_summary = preprocess(t5_cand)
# reference_summary1 = preprocess(by_human)
# reference_summary2 = preprocess(by_tarasha)
# reference_summary3 = preprocess(by_arjun)

# distance1 = model.wmdistance(model_summary, reference_summary1)
# print('distance = %.4f' % distance1)
# distance2 = model.wmdistance(model_summary, reference_summary2)
# print('distance = %.4f' % distance2)
# distance3 = model.wmdistance(model_summary, reference_summary3)
# print('distance = %.4f' % distance3)
# avg_dist = (distance1 + distance2 + distance3) / 3
# print('distance = %.4f' % avg_dist)

# Getting GPT2 model to evaluate text that is split by paragraph

In [ ]:
# from transformers import AutoTokenizer, GPT2LMHeadModel
# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
!pip install bert-extractive-summarizer
from summarizer import Summarizer,TransformerSummarizer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Based on ROUGE score

In [ ]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")

In [ ]:
# avg score from 1st summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_gpt2_by_human_rouge = {}

N = len(by_human)

for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  gpt2_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    gpt2_candidate.append(GPT2_model(para, min_length=50))
  gpt2_cand = ' '.join(gpt2_candidate)
  print(gpt2_cand)
    
  result = calc_rouge_scores(preprocess_to_str(gpt2_cand), preprocess_to_str(by_human[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

avg_gpt2_by_human_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_gpt2_by_human_rouge)

Recent indicators point to modest growth in spending and production. Job gains have been robust in recent months, and the unemployment rate has remained low. Russia’s war against Ukraine is causing tremendous human and economic hardship. The war and related e vents are contributing to upward pressure on inflation and are weighing on global economic activity. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run. The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary policy that is sufficiently restrictive to return inflation to 2 percent over time. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. The Committee’s assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation 

In [ ]:
# avg score from 2nd summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_gpt2_by_tarasha_rouge = {}

N = len(by_tarasha)

for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  gpt2_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    gpt2_candidate.append(GPT2_model(para, min_length=50))
  gpt2_cand = ' '.join(gpt2_candidate)
  print(gpt2_cand)
    
  result = calc_rouge_scores(preprocess_to_str(gpt2_cand), preprocess_to_str(by_tarasha[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

avg_gpt2_by_tarasha_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_gpt2_by_tarasha_rouge)

Recent indicators point to modest growth in spending and production. Job gains have been robust in recent months, and the unemployment rate has remained low. Russia’s war against Ukraine is causing tremendous human and economic hardship. The war and related e vents are contributing to upward pressure on inflation and are weighing on global economic activity. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run. The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary policy that is sufficiently restrictive to return inflation to 2 percent over time. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. The Committee’s assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation 

In [ ]:
# avg score from 3rd summaries
rouge1r, rouge1p, rouge1f = 0, 0, 0
rouge2r, rouge2p, rouge2f = 0, 0, 0
rougelr, rougelp, rougelf = 0, 0, 0
avg_gpt2_by_arjun_rouge = {}

N = len(by_arjun)

for i in range(N):  # looping through every text(splitted by paragraph) in the list 
  gpt2_candidate = []
  for para in by_para_mon[i]: # looping through every paragraph within that text
    gpt2_candidate.append(GPT2_model(para, min_length=50))
  gpt2_cand = ' '.join(gpt2_candidate)
  print(gpt2_cand)
    
  result = calc_rouge_scores(preprocess_to_str(gpt2_cand), preprocess_to_str(by_arjun[i]))
  rouge1r += result[0]['rouge-1']['r']
  rouge1p += result[0]['rouge-1']['p']
  rouge1f += result[0]['rouge-1']['f']

  rouge2r += result[0]['rouge-2']['r']
  rouge2p += result[0]['rouge-2']['p']
  rouge2f += result[0]['rouge-2']['f']

  rougelr += result[0]['rouge-l']['r']
  rougelp += result[0]['rouge-l']['p']
  rougelf += result[0]['rouge-l']['f']

avg_gpt2_by_arjun_rouge = {'rouge-1':{'r': rouge1r/N, 'p': rouge1p/N, 'f': rouge1f/N}, 'rouge-2':{'r': rouge2r/N, 'p': rouge2p/N, 'f': rouge2f/N}, 'rouge-l':{'r': rougelr/N, 'p': rougelp/N, 'f': rougelf/N}}
print(avg_gpt2_by_arjun_rouge)

Recent indicators point to modest growth in spending and production. Job gains have been robust in recent months, and the unemployment rate has remained low. Russia’s war against Ukraine is causing tremendous human and economic hardship. The war and related e vents are contributing to upward pressure on inflation and are weighing on global economic activity. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the l onger run. The Committee anticipates that ongoing increases in the target range will be appropriate in order to attain a stan ce of monetary policy that is sufficiently restrictive to return inflation to 2 percent over time. In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. The Committee’s assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation 

In [ ]:
# ## Combined avg of all 3 summaries
all_gpt2_rouge = [avg_gpt2_by_human_rouge, avg_gpt2_by_tarasha_rouge, avg_gpt2_by_arjun_rouge]

print(avg_rouge_results(all_gpt2_rouge))

{'rouge-1': {'r': 0.4561188727024958, 'p': 0.264443339227822, 'f': 0.32436219077582357}, 'rouge-2': {'r': 0.23575741736084432, 'p': 0.12097742229673047, 'f': 0.15470641532081944}, 'rouge-l': {'r': 0.4452518248993173, 'p': 0.25596816976127323, 'f': 0.31504689928312163}}


## Based on WMD

In [ ]:
# model_summary = preprocess(gpt2_cand)
# reference_summary1 = preprocess(by_human)
# reference_summary2 = preprocess(by_tarasha)
# reference_summary3 = preprocess(by_arjun)

# distance1 = model.wmdistance(model_summary, reference_summary1)
# print('distance = %.4f' % distance1)
# distance2 = model.wmdistance(model_summary, reference_summary2)
# print('distance = %.4f' % distance2)
# distance3 = model.wmdistance(model_summary, reference_summary3)
# print('distance = %.4f' % distance3)
# avg_dist = (distance1 + distance2 + distance3) / 3
# print('distance = %.4f' % avg_dist)

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("yijiyap/finscan_gpt2_test")
model = AutoModelWithLMHead.from_pretrained("yijiyap/finscan_gpt2_test")

/Users/yj/opt/anaconda3/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
print(len(model.transformer.h))

12


# Rankings based on rouge:
1. sshleifer-Bart (0.937)
2. gpt2 (0.481)
3. t5 (0.402)
4. pegasus (0.290)